In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
pd.set_option('max_colwidth',100)

In [2]:
## Using Selenium to get the travel time
## time.sleep(2) allows the website to load for 2 seconds 

def getTime(start_addr,dest_addr):
    driver = webdriver.Chrome()
    driver.maximize_window()
    # get the link of google map
    driver.get('https://www.google.com/maps')
    elem = driver.find_element_by_xpath('//*[@id="searchbox-directions"]') # click on the direction
    elem.click()
    time.sleep(2)
    input_from = driver.find_element_by_xpath('//*[@id="sb_ifc51"]/input')
    input_from.send_keys(start_addr) # input starting point
    time.sleep(2)
    input_to= driver.find_element_by_xpath('//*[@id="sb_ifc52"]/input')
    input_to.send_keys(dest_addr) #input destination point
    input_to.send_keys(Keys.RETURN)
    time.sleep(2)
    # select car buttom
    car = driver.find_element_by_xpath('//*[@id="omnibox-directions"]/div/div[2]/div/div/div[1]/div[2]/button/div[1]')
    car.click()
    time.sleep(2)
    travel_time = driver.find_element_by_xpath('//*[@id="section-directions-trip-0"]/div[2]/div[1]/div[1]/div[1]/span[1]')
    text = travel_time.get_attribute('textContent')
    driver.close()
    time_value = text.split(' ') # split ['6 mins'] into ['6' , 'mins']
    
    return int(time_value[0])

In [3]:
# main function

def get_plan():
    df = get_df()
    destination = df.DeliveryID.tolist() # destination list

    plan = [] # final delivery id plan
    curr = get_curr() #current Delivery id

    ## loop the funtion while destination list is not None

    while destination:
        max_value = float('-inf')
        max_index = float('-inf')
        start_addr = df.Address[df['DeliveryID'] == curr].to_string(index = False)# get address based on the delivery id = curr
        for i in destination:
            print("start_addr "+start_addr)
            dest_addr = df.Address[df['DeliveryID'] == i].to_string(index = False)# get address based on the delivery id = destination
            print("dest_addr "+dest_addr)
            time_value = getTime(start_addr,dest_addr) # get travel time from above getTime function
            print("value 1: "+ str(time_value))
            Value = float(df.Value[df['DeliveryID'] == i].to_string(index = False))# get Value based on the delivery id = destination
            Duration = float(df.Duration[df['DeliveryID'] == i].to_string(index = False))# get Duration based on the delivery id = destination
            Profit = Value - (Value/Duration)*(Duration + time_value)# calculate the profit
            print("value 2: "+ str(Profit))
            if Profit > max_value:
                max_value = Profit
                max_index = i
        print("max "+str(max_index))
        plan.append(max_index)
        destination.remove(max_index) # remove point with max profit
        curr = max_index
        
    ## prepare JSON for visualization
    temp = pd.DataFrame()
    temp['DeliveryID'] = plan
    add = []
    for i in plan:
        add.append(df.Address[df['DeliveryID'] == i].to_string(index = False))
    temp['Address'] = add

    return temp.to_json('plan.json',orient='index') 

,DeliveryID,Value,Duration,Start,End,Address
0,1,30,30,10,12,"11106 1/2 National Blvd, Los Angeles. CA 90034"
1,2,80,45,12,2,"Culver City, 10797 Washington Blvd, Culver City, CA 90232"
2,3,25,15,2,4,"10836 Washington Blvd, Culver City, CA 90232"
3,4,150,90,4,6,"3470 S Sepulveda Blvd, Los Angeles, CA 90034"


In [4]:
def get_curr():
    current = '4331 Overland Ave, Culver City'
    return curr

In [5]:
df = pd.read_csv('Project1.csv')
def get_df():
    df = pd.read_csv('Project1.csv')
    return df


,DeliveryID,Value,Duration,Start,End,Address,status
0,1,30,30,10,12,"11106 1/2 National Blvd, Los Angeles. CA 90034",completed
1,2,80,45,12,2,"Culver City, 10797 Washington Blvd, Culver City, CA 90232",completed
2,3,25,15,2,4,"10836 Washington Blvd, Culver City, CA 90232",incomplete
3,4,150,90,4,6,"3470 S Sepulveda Blvd, Los Angeles, CA 90034",incomplete
